# Lesson 09: Quadratic Discriminant Analysis (QDA)## Objectives- Implement QDA with class-specific covariances.- Visualize quadratic decision boundaries.- Compare QDA to LDA on the same data.

## From the notesQDA assumes \(p(x\mid y=k)\) is Gaussian with class-specific covariance \(\Sigma_k\).The resulting boundary is quadratic.

## IntuitionAllowing \(\Sigma_k\) to vary gives more flexible decision boundaries at the cost of more parameters.

## DataWe generate two classes with different covariance shapes.

In [ ]:
import numpy as npimport matplotlib.pyplot as pltnp.random.seed(42)

In [ ]:
# Synthetic data with different covariancesm = 140mean0 = np.array([-1.0, 0.0])mean1 = np.array([1.5, 1.0])Sigma0 = np.array([[0.7, 0.2],[0.2, 0.4]])Sigma1 = np.array([[0.4, -0.1],[-0.1, 0.8]])X0 = np.random.multivariate_normal(mean0, Sigma0, size=m//2)X1 = np.random.multivariate_normal(mean1, Sigma1, size=m//2)X = np.vstack([X0, X1])y = np.array([0]*(m//2) + [1]*(m//2))

## Implementation: QDA

In [ ]:
def qda_fit(X, y):    classes = np.unique(y)    means = {}    covs = {}    priors = {}    for c in classes:        Xc = X[y == c]        means[c] = Xc.mean(axis=0)        covs[c] = np.cov(Xc.T)        priors[c] = len(Xc) / len(X)    return means, covs, priorsdef qda_predict(X, means, covs, priors):    scores = []    for c in sorted(means.keys()):        mu = means[c]        Sigma = covs[c]        Sigma_inv = np.linalg.pinv(Sigma)        det = np.linalg.det(Sigma)        diffs = X - mu        quad = np.sum(diffs @ Sigma_inv * diffs, axis=1)        score = -0.5 * (np.log(det + 1e-9) + quad) + np.log(priors[c])        scores.append(score)    scores = np.vstack(scores).T    return np.argmax(scores, axis=1)

## Experiments

In [ ]:
means, covs, priors = qda_fit(X, y)preds = qda_predict(X, means, covs, priors)acc = (preds == y).mean()print(f"Accuracy: {acc:.2f}")

## Visualizations

In [ ]:
plt.figure(figsize=(6,4))plt.scatter(X0[:,0], X0[:,1], label="class 0", alpha=0.7)plt.scatter(X1[:,0], X1[:,1], label="class 1", alpha=0.7)plt.xlabel("x1")plt.ylabel("x2")plt.title("QDA data")plt.legend()plt.show()# Decision regionsx1_vals = np.linspace(X[:,0].min()-1, X[:,0].max()+1, 200)x2_vals = np.linspace(X[:,1].min()-1, X[:,1].max()+1, 200)xx1, xx2 = np.meshgrid(x1_vals, x2_vals)X_grid = np.c_[xx1.ravel(), xx2.ravel()]Z = qda_predict(X_grid, means, covs, priors).reshape(xx1.shape)plt.figure(figsize=(6,4))plt.contourf(xx1, xx2, Z, alpha=0.3, levels=2)plt.scatter(X0[:,0], X0[:,1], label="class 0", alpha=0.7)plt.scatter(X1[:,0], X1[:,1], label="class 1", alpha=0.7)plt.title("QDA decision regions")plt.xlabel("x1")plt.ylabel("x2")plt.legend()plt.show()

## Takeaways- QDA allows flexible decision boundaries by modeling class-specific covariance.- It can overfit with limited data.

## Explain it in an interview- Highlight the difference between LDA and QDA assumptions.- Mention quadratic boundaries and parameter count.

## Exercises1. Compare QDA vs LDA accuracy on this dataset.2. Add a regularization term to covariance estimates.3. Visualize the log-likelihood contours for each class.